<a href="https://colab.research.google.com/github/DanielMerkdo/analitica_marketing/blob/main/c_modelado_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CARGUE Y LECTURA TABLA FINAL

In [ ]:
# Montar drive y crear path para sys y para directorio de trabajo

from google.colab import drive
drive.mount('/content/drive')

import sys
import os

# Define el path al directorio de trabajo actual
path = "/content/drive/My Drive/cod/analitica_marketing"


sys.path.append(path) ### para importar archivo de funciones propias a través de import

os.chdir(path) ### para que por defecto suba y descargue archivos partiendo de esa ruta

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 ### Importar librerias
 import numpy as np
 import pandas as pd
 import sqlite3 as sql
 from sklearn.preprocessing import MinMaxScaler
 from ipywidgets import interact
 from sklearn import neighbors
 import joblib

In [ ]:
 ### Conectarse con la base de datos
 conex= sql.connect('salida/db_movies_final.db')
 cur= conex.cursor()

In [ ]:
 ### Ver las tablas que hay en la base de datos
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('db_movies_final',)]

In [ ]:
column_names_query = "SELECT name FROM PRAGMA_TABLE_INFO('db_movies_final')"
columns_ratings = pd.read_sql(column_names_query, conex)
columns_ratings


,name
0,user_id
1,movie_Id
2,movie_rating
3,rating_year
4,movie_title
5,movie_year
6,Action
7,Adventure
8,Animation
9,Children


In [ ]:
final_movies= pd.read_sql("""SELECT * FROM db_movies_final""", conex)
final_movies

,user_id,movie_Id,movie_rating,rating_year,movie_title,movie_year,Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,2000,Toy Story,1995,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,2000,Grumpier Old Men,1995,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,2000,Heat,1995,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,2000,Seven (a.k.a. Se7en),1995,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,2000,"Usual Suspects, The",1995,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48757,609,828,3.0,1996,"Adventures of Pinocchio, The",1996,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
48758,609,833,3.0,1996,High School High,1996,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48759,609,892,3.0,1996,Twelfth Night,1996,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
48760,609,1056,3.0,1996,Jude,1996,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#SISTEMAS BASADOS EN POPULARIDAD



In [ ]:
### 10 Mejores peliculas calificadas que no tengan nunca una calificacion de 0.0 y año de la pelicula mayor a 1990, y que hayan sido rankeadas mas de 100 veces
pd.read_sql("""select movie_title, movie_year,
            avg(movie_rating) as avg_rat,
            count(*) as count_rating
            from db_movies_final
            where movie_rating<>0.0
             AND movie_year > 1990
            group by movie_title
            having count_rating >100
            order by avg_rat desc
            limit 10
            """, conex)

,movie_title,movie_year,avg_rat,count_rating
0,"Shawshank Redemption, The",1994,4.442804,271
1,Fight Club,1999,4.276786,168
2,"Usual Suspects, The",1995,4.256410,156
3,"Dark Knight, The",2008,4.252174,115
4,Schindler's List,1993,4.216667,180
5,Forrest Gump,1994,4.210037,269
6,"Matrix, The",1999,4.207763,219
7,Fargo,1996,4.170455,132
8,"Silence of the Lambs, The",1991,4.163043,230
9,Pulp Fiction,1994,4.155512,254


In [ ]:
### La peliculas mas vistas con promedio
pd.read_sql("""
    SELECT movie_title,movie_year,
           AVG(CASE WHEN movie_rating = 0 THEN NULL ELSE movie_rating END) AS avg_cal,
           COUNT(*) AS count_rating
    FROM db_movies_final
    GROUP BY movie_title
    ORDER BY count_rating DESC
    LIMIT 10
""", conex)

,movie_title,movie_year,avg_cal,count_rating
0,"Shawshank Redemption, The",1994,4.442804,271
1,Forrest Gump,1994,4.210037,269
2,Pulp Fiction,1994,4.155512,254
3,"Silence of the Lambs, The",1991,4.163043,230
4,"Matrix, The",1999,4.207763,219
5,Braveheart,1995,4.058974,195
6,Star Wars: Episode IV - A New Hope,1977,4.204663,193
7,Jurassic Park,1993,3.759459,185
8,Schindler's List,1993,4.216667,180
9,Terminator 2: Judgment Day,1991,3.971429,175


#SISTEMA DE RECOMENDACIÓN BASADO EN CONTENIDO DE UN PRODUCTO

In [ ]:
final_movies2= pd.read_sql("""select * from db_movies_final""", conex)
final_movies2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48762 entries, 0 to 48761
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       48762 non-null  int64  
 1   movie_Id      48762 non-null  int64  
 2   movie_rating  48762 non-null  float64
 3   rating_year   48762 non-null  int64  
 4   movie_title   48762 non-null  object 
 5   movie_year    48762 non-null  int64  
 6   Action        48762 non-null  int64  
 7   Adventure     48762 non-null  int64  
 8   Animation     48762 non-null  int64  
 9   Children      48762 non-null  int64  
 10  Comedy        48762 non-null  int64  
 11  Crime         48762 non-null  int64  
 12  Documentary   48762 non-null  int64  
 13  Drama         48762 non-null  int64  
 14  Fantasy       48762 non-null  int64  
 15  Film-Noir     48762 non-null  int64  
 16  Horror        48762 non-null  int64  
 17  IMAX          48762 non-null  int64  
 18  Musical       48762 non-nu

In [ ]:
##### escalar para que año esté en el mismo rango ###
sc=MinMaxScaler()
final_movies2[["movie_yearsc"]]=sc.fit_transform(final_movies2[['movie_year']])



In [ ]:
final_moviesknn=final_movies2.drop(columns=['user_id','movie_Id','movie_rating','movie_year','rating_year','movie_title'])


In [ ]:
## el coseno de un angulo entre dos vectores es 1 cuando son perpendiculares y 0 cuando son paralelos(indicando que son muy similar324e-06	3.336112e-01	3.336665e-01	3.336665e-es)
model = neighbors.NearestNeighbors(n_neighbors=20, metric='cosine')
model.fit(final_moviesknn)
dist, idlist = model.kneighbors(final_moviesknn)

In [ ]:
distancias=pd.DataFrame(dist) ## devuelve un ranking de la distancias más cercanas para cada fila(libro)
id_list=pd.DataFrame(idlist) ## para saber esas distancias a que item corresponde

In [ ]:
####ejemplo para una pelicula
movies_list_name = []
movie_name='Forrest Gump'
movie_id = db_movies_final[db_movies_final['movie_title'] == movie_name].index ### extraer el indice del libro
movie_id = movie_id[0] ## si encuentra varios solo guarde uno

for newid in idlist[movie_id]:
        movies_list_name.append(db_movies_final.loc[newid].movie_title) ### agrega el nombre de cada una de los id recomendados

movies_list_name

NameError: name 'db_movies_final' is not defined

In [ ]:
def MovieRecomender(movie_name= list(db_movies_final["movie_title"].value_counts().index)):
    movie_list_name= []
    movie_id= db_movies_final[db_movies_final["movie_title"]== movie_name].index
    movie_id= movie_id[0]
    for newid in idlist[movie_id]:
      movie_list_name.append(db_movies_final.loc[newid].movie_title)
    return movie_list_name

NameError: name 'db_movies_final' is not defined

In [ ]:
print(interact(MovieRecomender))

interactive(children=(Dropdown(description='movie_name', options=('Shawshank Redemption, The', 'Forrest Gump',…

<function MovieRecomender at 0x7b8207cd1f80>
